### Capturing images from a stereo camera

In [1]:
import numpy as np
import cv2
import time
import os

res_fac = 1 #each of the spatial dimensions of the frames will be res_fac(resize_factor) smaller

lcam = cv2.VideoCapture(1)
rcam = cv2.VideoCapture(0)

i = 0

while(True):
    sl, fl = lcam.read()
    sr, fr = rcam.read()
    
    #assuming identical resolution
    h, w, _ = fl.shape
    
    fl = cv2.rotate(fl, cv2.ROTATE_180)
    fr = cv2.rotate(fr, cv2.ROTATE_180)
    
    if (sl == False or sr == False):
        print("Cannot read frame")
        
        break
    
    key = cv2.waitKey(70) & 0xFF
    
    if (key == ord('q')):
        break
    
    elif (key == 32): #space
        cv2.imwrite("calib/" + str(i) + "_l.jpg", fl)
        cv2.imwrite("calib/" + str(i) + "_r.jpg", fr)

        i += 1
    
    flres = cv2.resize(fl, (int(w / res_fac), int(h / res_fac)))
    frres = cv2.resize(fr, (int(w / res_fac), int(h / res_fac)))
    
    result = np.concatenate((flres, frres), axis=1)
    
    cv2.imshow("frames", result)
    #cv2.imshow("left", fl)
    #cv2.imshow("right", fr)

lcam.release()
rcam.release()

cv2.destroyAllWindows()
cv2.waitKey(10)

-1

### Stereo pair calibration

- Intrinsic_mtx_1, dist_1 - intrinsics and distortion of the first camera
- Intrinsic_mtx_2, dist_2 - intrinsics and distortion of the second camera

- R - rotation matrix between cameras
- T - translation matrix between cameras

https://github.com/bvnayak/stereo_calibration.git

In [1]:
import numpy as np
import cv2
import glob
import argparse

class StereoCalibration(object):
    def __init__(self, filepath):
        # termination criteria
        self.criteria = (cv2.TERM_CRITERIA_EPS +
                         cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        self.criteria_cal = (cv2.TERM_CRITERIA_EPS +
                             cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        self.objp = np.zeros((8*6, 3), np.float32)
        self.objp[:, :2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2) * 0.027

        # Arrays to store object points and image points from all the images.
        self.objpoints = []  # 3d point in real world space
        self.imgpoints_l = []  # 2d points in image plane.
        self.imgpoints_r = []  # 2d points in image plane.

        self.cal_path = filepath
        self.read_images(self.cal_path)

    def read_images(self, cal_path):
        images_right = glob.glob(cal_path + '*_r.jpg')
        images_left = glob.glob(cal_path + '*_l.jpg')
        images_left.sort()
        images_right.sort()

        print ("images_left", images_left, images_right)

        for i, fname in enumerate(images_right):
            img_l = cv2.imread(images_left[i])
            img_r = cv2.imread(images_right[i])

            gray_l = cv2.cvtColor(img_l, cv2.COLOR_BGR2GRAY)
            gray_r = cv2.cvtColor(img_r, cv2.COLOR_BGR2GRAY)

            # Find the chess board corners
            ret_l, corners_l = cv2.findChessboardCorners(gray_l, (8, 6), None)
            ret_r, corners_r = cv2.findChessboardCorners(gray_r, (8, 6), None)

            # If found, add object points, image points (after refining them)
            self.objpoints.append(self.objp)

            if ret_l is True:
                rt = cv2.cornerSubPix(gray_l, corners_l, (11, 11),
                                      (-1, -1), self.criteria)
                self.imgpoints_l.append(corners_l)

                # Draw and display the corners
                ret_l = cv2.drawChessboardCorners(img_l, (8, 6),
                                                  corners_l, ret_l)
                cv2.imshow(images_left[i], img_l)
                cv2.waitKey(500)

            if ret_r is True:
                rt = cv2.cornerSubPix(gray_r, corners_r, (11, 11),
                                      (-1, -1), self.criteria)
                self.imgpoints_r.append(corners_r)

                # Draw and display the corners
                ret_r = cv2.drawChessboardCorners(img_r, (8, 6),
                                                  corners_r, ret_r)
                cv2.imshow(images_right[i], img_r)
                cv2.waitKey(100)
            img_shape = gray_l.shape[::-1]

        rt, self.M1, self.d1, self.r1, self.t1 = cv2.calibrateCamera(
            self.objpoints, self.imgpoints_l, img_shape, None, None)
        rt, self.M2, self.d2, self.r2, self.t2 = cv2.calibrateCamera(
            self.objpoints, self.imgpoints_r, img_shape, None, None)

        self.camera_model = self.stereo_calibrate(img_shape)

    def stereo_calibrate(self, dims):
        flags = 0
        flags |= cv2.CALIB_FIX_INTRINSIC
        # flags |= cv2.CALIB_FIX_PRINCIPAL_POINT
        flags |= cv2.CALIB_USE_INTRINSIC_GUESS
        flags |= cv2.CALIB_FIX_FOCAL_LENGTH
        # flags |= cv2.CALIB_FIX_ASPECT_RATIO
        flags |= cv2.CALIB_ZERO_TANGENT_DIST
        # flags |= cv2.CALIB_RATIONAL_MODEL
        # flags |= cv2.CALIB_SAME_FOCAL_LENGTH
        # flags |= cv2.CALIB_FIX_K3
        # flags |= cv2.CALIB_FIX_K4
        # flags |= cv2.CALIB_FIX_K5

        stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER +
                                cv2.TERM_CRITERIA_EPS, 100, 1e-5)
        ret, M1, d1, M2, d2, R, T, E, F = cv2.stereoCalibrate(
            self.objpoints, self.imgpoints_l,
            self.imgpoints_r, self.M1, self.d1, self.M2,
            self.d2, dims,
            criteria=stereocalib_criteria, flags=flags)

        print('Intrinsic_mtx_1', M1)
        print('dist_1', d1)
        print('Intrinsic_mtx_2', M2)
        print('dist_2', d2)
        print('R', R)
        print('T', T)
        print('E', E)
        print('F', F)

        # for i in range(len(self.r1)):
        #     print("--- pose[", i+1, "] ---")
        #     self.ext1, _ = cv2.Rodrigues(self.r1[i])
        #     self.ext2, _ = cv2.Rodrigues(self.r2[i])
        #     print('Ext1', self.ext1)
        #     print('Ext2', self.ext2)

        print('')

        camera_model = dict([('M1', M1), ('M2', M2), ('dist1', d1),
                            ('dist2', d2), ('rvecs1', self.r1),
                            ('rvecs2', self.r2), ('R', R), ('T', T),
                            ('E', E), ('F', F)])

        cv2.destroyAllWindows()
        return camera_model

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('filepath', help='String Filepath')
#     args = parser.parse_args()
#     cal_data = StereoCalibration(args.filepath)



In [2]:
#from stereo_calibration.camera_calibrate import StereoCalibration
cal = StereoCalibration('calib_s/')
cal.camera_model

images_left ['calib_s\\0_l.jpg', 'calib_s\\1_l.jpg', 'calib_s\\2_l.jpg', 'calib_s\\3_l.jpg', 'calib_s\\4_l.jpg', 'calib_s\\5_l.jpg', 'calib_s\\6_l.jpg', 'calib_s\\7_l.jpg', 'calib_s\\8_l.jpg', 'calib_s\\9_l.jpg'] ['calib_s\\0_r.jpg', 'calib_s\\1_r.jpg', 'calib_s\\2_r.jpg', 'calib_s\\3_r.jpg', 'calib_s\\4_r.jpg', 'calib_s\\5_r.jpg', 'calib_s\\6_r.jpg', 'calib_s\\7_r.jpg', 'calib_s\\8_r.jpg', 'calib_s\\9_r.jpg']
Intrinsic_mtx_1 [[419.1283445    0.         312.59169907]
 [  0.         556.24148484 276.49586824]
 [  0.           0.           1.        ]]
dist_1 [[-0.26061381  0.00100663  0.0028365   0.00226141 -0.04798144]]
Intrinsic_mtx_2 [[422.25695492   0.         328.98016472]
 [  0.         562.23479307 263.74863412]
 [  0.           0.           1.        ]]
dist_2 [[-0.21382583 -0.15528625  0.00273781 -0.00160017 -0.12728611]]
R [[ 0.9998229  -0.00622574 -0.01775985]
 [ 0.0064956   0.99986371  0.01517811]
 [ 0.01766293 -0.01529078  0.99972707]]
T [[ 0.10606028]
 [-0.00148929]
 [ 0.0

{'M1': array([[419.1283445 ,   0.        , 312.59169907],
        [  0.        , 556.24148484, 276.49586824],
        [  0.        ,   0.        ,   1.        ]]),
 'M2': array([[422.25695492,   0.        , 328.98016472],
        [  0.        , 562.23479307, 263.74863412],
        [  0.        ,   0.        ,   1.        ]]),
 'dist1': array([[-0.26061381,  0.00100663,  0.0028365 ,  0.00226141, -0.04798144]]),
 'dist2': array([[-0.21382583, -0.15528625,  0.00273781, -0.00160017, -0.12728611]]),
 'rvecs1': (array([[-0.79089952],
         [ 0.47640131],
         [ 0.44465695]]),
  array([[-0.18711017],
         [ 0.52005061],
         [ 0.2931456 ]]),
  array([[-0.09005569],
         [ 0.63451767],
         [ 0.41842142]]),
  array([[ 0.49369702],
         [-0.82035279],
         [ 0.52586018]]),
  array([[-0.04858184],
         [ 0.54246106],
         [-0.36892171]]),
  array([[-0.77691309],
         [ 0.32675707],
         [ 0.10486113]]),
  array([[-0.10231676],
         [ 0.59246446]

### Frames rectification

In [3]:
new_mtxR = cal.camera_model["M1"]
new_mtxL = cal.camera_model["M2"]
distL = cal.camera_model["dist1"]
distR = cal.camera_model["dist2"]
R = cal.camera_model["R"]
T = cal.camera_model["T"]

imgL = cv2.imread("calib_s/3_l.jpg")
imgR = cv2.imread("calib_s/3_r.jpg")

imgL_gray = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
imgR_gray = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)

rectify_scale = 2
rect_l, rect_r, proj_mat_l, proj_mat_r, Q, roiL, roiR = cv2.stereoRectify(new_mtxL,
    distL, new_mtxR, distR, imgL_gray.shape[::-1], R, T, rectify_scale, (0,0))

Left_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxL, distL, rect_l, proj_mat_l,
                                             imgL_gray.shape[::-1], cv2.CV_16SC2)
Right_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxR, distR, rect_r, proj_mat_r,
                                              imgR_gray.shape[::-1], cv2.CV_16SC2)

Left_nice= cv2.remap(imgL,Left_Stereo_Map[0],Left_Stereo_Map[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
Right_nice= cv2.remap(imgR,Right_Stereo_Map[0],Right_Stereo_Map[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
 
cv2.imshow("Left image after rectification", Left_nice)
cv2.imshow("Right image after rectification", Right_nice)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

### Saving maps to the disc

In [6]:
cv_file = cv2.FileStorage("improved_params2.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("Left_Stereo_Map_x",Left_Stereo_Map[0])
cv_file.write("Left_Stereo_Map_y",Left_Stereo_Map[1])
cv_file.write("Right_Stereo_Map_x",Right_Stereo_Map[0])
cv_file.write("Right_Stereo_Map_y",Right_Stereo_Map[1])
cv_file.release()

In [4]:
import cv2

cv_file = cv2.FileStorage("improved_params2.xml", cv2.FILE_STORAGE_READ)
Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()

cv_file.release()

In [25]:
pip uninstall opencv-python

^C
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


### Disparity map obtainment

https://learnopencv.com/depth-perception-using-stereo-camera-python-c/

In [2]:
import numpy as np
import cv2

res_fac = 2

CamL = cv2.VideoCapture(1)
CamR = cv2.VideoCapture(0)

# Left_Stereo_Map_x = Left_Stereo_Map[0]
# Left_Stereo_Map_y = Left_Stereo_Map[1]
# Right_Stereo_Map_x = Right_Stereo_Map[0]
# Right_Stereo_Map_y = Right_Stereo_Map[1]

def nothing(x):
    pass
 
cv2.namedWindow('disp',cv2.WINDOW_NORMAL)

#cv2.resizeWindow('disp',600,600)

win_size = 5
min_disp = -1
max_disp = 63 #min_disp * 9
num_disp = max_disp - min_disp # Needs to be divisible by 16

blockSize = 3#9
uniquenessRatio = 5
speckleWindowSize = 5
speckleRange = 5
disp12MaxDiff = 2
P1 = 8*3*win_size**2
P2 =32*3*win_size**2

cv2.createTrackbar('numDisparities','disp',14,37,nothing)
cv2.createTrackbar('blockSize','disp',4,50,nothing)
cv2.createTrackbar('preFilterType','disp',1,1,nothing)
cv2.createTrackbar('preFilterSize','disp',3,25,nothing)
cv2.createTrackbar('preFilterCap','disp',5,62,nothing)
cv2.createTrackbar('textureThreshold','disp',10,100,nothing)

cv2.createTrackbar('uniquenessRatio','disp',0,100,nothing)
cv2.createTrackbar('speckleRange','disp',0,100,nothing)
cv2.createTrackbar('speckleWindowSize','disp',0,25,nothing)
cv2.createTrackbar('disp12MaxDiff','disp',disp12MaxDiff,25,nothing)
cv2.createTrackbar('minDisparity','disp',1,25,nothing)

# Creating an object of StereoBM algorithm
stereo = cv2.StereoBM_create()

# print(Left_Stereo_Map_x, Left_Stereo_Map_y)

# print(Right_Stereo_Map_x, Right_Stereo_Map_y)

while True:
    # Capturing and storing left and right camera images
    retL, imgL= CamL.read()
    retR, imgR= CamR.read()
   
    imgL = cv2.rotate(imgL, cv2.ROTATE_180)
    imgR = cv2.rotate(imgR, cv2.ROTATE_180)

    #print(imgL, imgR)

    # Proceed only if the frames have been captured
    if retL and retR:
        imgR_gray = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
        imgL_gray = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)

    # Applying stereo image rectification on the left image
    Left_nice= cv2.remap(imgL_gray,
              Left_Stereo_Map_x,
              Left_Stereo_Map_y,
              cv2.INTER_LANCZOS4,
              cv2.BORDER_CONSTANT,
              0)
     
    # Applying stereo image rectification on the right image
    Right_nice= cv2.remap(imgR_gray,
              Right_Stereo_Map_x,
              Right_Stereo_Map_y,
              cv2.INTER_LANCZOS4,
              cv2.BORDER_CONSTANT,
              0)
 
    # Updating the parameters based on the trackbar positions
    numDisparities = cv2.getTrackbarPos('numDisparities','disp')*16
    blockSize = cv2.getTrackbarPos('blockSize','disp')*2 + 5
    preFilterType = cv2.getTrackbarPos('preFilterType','disp')
    preFilterSize = cv2.getTrackbarPos('preFilterSize','disp')*2 + 5
    preFilterCap = cv2.getTrackbarPos('preFilterCap','disp')
    textureThreshold = cv2.getTrackbarPos('textureThreshold','disp')
    uniquenessRatio = cv2.getTrackbarPos('uniquenessRatio','disp')
    speckleRange = cv2.getTrackbarPos('speckleRange','disp')
    speckleWindowSize = cv2.getTrackbarPos('speckleWindowSize','disp')*2
    disp12MaxDiff = cv2.getTrackbarPos('disp12MaxDiff','disp')
    minDisparity = cv2.getTrackbarPos('minDisparity','disp') - 2
     
    # Setting the updated parameters before computing disparity map
    stereo.setNumDisparities(numDisparities)
    stereo.setBlockSize(blockSize)
    stereo.setPreFilterType(preFilterType)
    stereo.setPreFilterSize(preFilterSize)
    stereo.setPreFilterCap(preFilterCap)
    stereo.setTextureThreshold(textureThreshold)
    stereo.setUniquenessRatio(uniquenessRatio)
    stereo.setSpeckleRange(speckleRange)
    stereo.setSpeckleWindowSize(speckleWindowSize)
    stereo.setDisp12MaxDiff(disp12MaxDiff)
    stereo.setMinDisparity(minDisparity)
 
    # Calculating disparity using the StereoBM algorithm
    
    h, w = Left_nice.shape
    
    flres = cv2.resize(Left_nice, (int(w / res_fac), int(h / res_fac)))
    frres = cv2.resize(Right_nice, (int(w / res_fac), int(h / res_fac)))

    disparity = stereo.compute(flres, frres)
    
    #print(flres.shape, disparity.shape)
    
    # NOTE: Code returns a 16bit signed single channel image,
    # CV_16S containing a disparity map scaled by 16. Hence it 
    # is essential to convert it to CV_32F and scale it down 16 times.
 
    # Converting to float32 
    #########disparity = disparity.astype(np.float32)
 
    sigma = 1.5
    lmbda = 8000.0
    #left_matcher = cv2.StereoBM_create(max_disp, wsize);
    right_matcher = cv2.ximgproc.createRightMatcher(stereo)
    #left_disp = left_matcher.compute(left_image, right_image);
    right_disp = right_matcher.compute(frres,flres)

    # Now create DisparityWLSFilter
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(stereo)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)
    filtered_disp = wls_filter.filter(disparity, flres, disparity_map_right=right_disp)

    # Scaling down the disparity values and normalizing them 
    filt_disp = (filtered_disp/16.0 - minDisparity)/numDisparities
 
    # Displaying the disparity map
    cv2.imshow("disparity", disparity)

    cv2.imshow("filtered disparity", filt_disp)

    cv2.imshow("flres", flres)
 
    # Close window using esc key
    key = cv2.waitKey(300) & 0xFF
    
    if (key == ord('q')):
        break

CamL.release()
CamR.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

-1

### Visualizing point cloud

In [4]:
import numpy as np
import cv2
import open3d
import time

def vis (X, color = np.array([0,0,1], dtype='float64')):
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(X)
    pcd.paint_uniform_color(color)
    
    vis_transform = [[1, 0, 0, 0],
                 [0,-1, 0, 0],
                 [0, 0,-1, 0],
                 [0, 0, 0, 1]] 
    
    pcd.transform(vis_transform)
    
    open3d.visualization.draw_geometries([pcd])

res_fac = 2

CamL = cv2.VideoCapture(1)
CamR = cv2.VideoCapture(0)

# Left_Stereo_Map_x = Left_Stereo_Map[0]
# Left_Stereo_Map_y = Left_Stereo_Map[1]
# Right_Stereo_Map_x = Right_Stereo_Map[0]
# Right_Stereo_Map_y = Right_Stereo_Map[1]

def nothing(x):
    pass
 
cv2.namedWindow('disp',cv2.WINDOW_NORMAL)

#cv2.resizeWindow('disp',600,600)

win_size = 5
min_disp = -1
max_disp = 63 #min_disp * 9
num_disp = max_disp - min_disp # Needs to be divisible by 16

blockSize = 3#9
uniquenessRatio = 5
speckleWindowSize = 5
speckleRange = 5
disp12MaxDiff = 2
P1 = 8*3*win_size**2
P2 =32*3*win_size**2

cv2.createTrackbar('numDisparities','disp',14,37,nothing)
cv2.createTrackbar('blockSize','disp',4,50,nothing)
cv2.createTrackbar('preFilterType','disp',1,1,nothing)
cv2.createTrackbar('preFilterSize','disp',3,25,nothing)
cv2.createTrackbar('preFilterCap','disp',5,62,nothing)
cv2.createTrackbar('textureThreshold','disp',10,100,nothing)

cv2.createTrackbar('uniquenessRatio','disp',0,100,nothing)
cv2.createTrackbar('speckleRange','disp',0,100,nothing)
cv2.createTrackbar('speckleWindowSize','disp',0,25,nothing)
cv2.createTrackbar('disp12MaxDiff','disp',disp12MaxDiff,25,nothing)
cv2.createTrackbar('minDisparity','disp',1,25,nothing)

# Creating an object of StereoBM algorithm
stereo = cv2.StereoBM_create()

cv_file = cv2.FileStorage("improved_params2.xml", cv2.FILE_STORAGE_READ)
Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

# print(Left_Stereo_Map_x, Left_Stereo_Map_y)
# print(Right_Stereo_Map_x, Right_Stereo_Map_y)

vis = open3d.visualization.Visualizer()
vis.create_window()

#pcd = o3d.io.read_point_cloud("reconstructed.ply")

pcd = open3d.geometry.PointCloud()
vis.add_geometry(pcd)

#vis.create_window()

vis_transform = [[1, 0, 0, 0],
                 [0,-1, 0, 0],
                 [0, 0,-1, 0],
                 [0, 0, 0, 1]] # transforms to make viewpoint match camera perspective

while True:
    # Capturing and storing left and right camera images
    retL, imgL= CamL.read()
    retR, imgR= CamR.read()
   
    imgL = cv2.rotate(imgL, cv2.ROTATE_180)
    imgR = cv2.rotate(imgR, cv2.ROTATE_180)

    #print(imgL, imgR)

    # Proceed only if the frames have been captured
    if retL and retR:
        imgR_gray = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
        imgL_gray = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)

    # Applying stereo image rectification on the left image
    Left_nice= cv2.remap(imgL_gray,
              Left_Stereo_Map_x,
              Left_Stereo_Map_y,
              cv2.INTER_LANCZOS4,
              cv2.BORDER_CONSTANT,
              0)
     
    # Applying stereo image rectification on the right image
    Right_nice= cv2.remap(imgR_gray,
              Right_Stereo_Map_x,
              Right_Stereo_Map_y,
              cv2.INTER_LANCZOS4,
              cv2.BORDER_CONSTANT,
              0)
 
    # Updating the parameters based on the trackbar positions
    numDisparities = cv2.getTrackbarPos('numDisparities','disp')*16
    blockSize = cv2.getTrackbarPos('blockSize','disp')*2 + 5
    preFilterType = cv2.getTrackbarPos('preFilterType','disp')
    preFilterSize = cv2.getTrackbarPos('preFilterSize','disp')*2 + 5
    preFilterCap = cv2.getTrackbarPos('preFilterCap','disp')
    textureThreshold = cv2.getTrackbarPos('textureThreshold','disp')
    uniquenessRatio = cv2.getTrackbarPos('uniquenessRatio','disp')
    speckleRange = cv2.getTrackbarPos('speckleRange','disp')
    speckleWindowSize = cv2.getTrackbarPos('speckleWindowSize','disp')*2
    disp12MaxDiff = cv2.getTrackbarPos('disp12MaxDiff','disp')
    minDisparity = cv2.getTrackbarPos('minDisparity','disp') - 2
     
    # Setting the updated parameters before computing disparity map
    stereo.setNumDisparities(numDisparities)
    stereo.setBlockSize(blockSize)
    stereo.setPreFilterType(preFilterType)
    stereo.setPreFilterSize(preFilterSize)
    stereo.setPreFilterCap(preFilterCap)
    stereo.setTextureThreshold(textureThreshold)
    stereo.setUniquenessRatio(uniquenessRatio)
    stereo.setSpeckleRange(speckleRange)
    stereo.setSpeckleWindowSize(speckleWindowSize)
    stereo.setDisp12MaxDiff(disp12MaxDiff)
    stereo.setMinDisparity(minDisparity)
 
    # Calculating disparity using the StereoBM algorithm
    
    h, w = Left_nice.shape
    
    flres = cv2.resize(Left_nice, (int(w / res_fac), int(h / res_fac)))
    frres = cv2.resize(Right_nice, (int(w / res_fac), int(h / res_fac)))

    disparity = stereo.compute(flres, frres)
    
    #print(flres.shape, disparity.shape)
    
    # NOTE: Code returns a 16bit signed single channel image,
    # CV_16S containing a disparity map scaled by 16. Hence it 
    # is essential to convert it to CV_32F and scale it down 16 times.
 
    # Converting to float32 
    #########disparity = disparity.astype(np.float32)
 
    sigma = 1.5
    lmbda = 8000.0
    #left_matcher = cv2.StereoBM_create(max_disp, wsize);
    right_matcher = cv2.ximgproc.createRightMatcher(stereo);
    #left_disp = left_matcher.compute(left_image, right_image);
    right_disp = right_matcher.compute(frres,flres);

    # Now create DisparityWLSFilter
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(stereo);
    wls_filter.setLambda(lmbda);
    wls_filter.setSigmaColor(sigma);
    filtered_disp = wls_filter.filter(disparity, flres, disparity_map_right=right_disp)

    # Scaling down the disparity values and normalizing them 
    filt_disp = (filtered_disp/16.0 - minDisparity)/numDisparities
 
    # Displaying the disparity map
    #cv2.imshow("dispar", disparity)

    cv2.imshow("fildispar", filt_disp)

    scaled = (filt_disp * 256).astype(np.uint8)

    cv2.imshow("sdcsd", scaled)
        
    points_3D = cv2.reprojectImageTo3D(scaled, Q)

    #Get color points
    #colors = cv2.cvtColor(flres, cv2.COLOR_BGR2RGB)

    #Get rid of points with value 0 (i.e no depth)
    mask_map = scaled > 70#disparity.min()

    #Mask colors and points. 
    output_points = points_3D[mask_map]
    #output_colors = colors[mask_map]
    
    vis.remove_geometry(pcd,False)

    skip = 50
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(output_points[::skip])
    
    colors = cv2.cvtColor(cv2.resize(imgL, (int(w / res_fac), int(h / res_fac))), cv2.COLOR_BGR2RGB)
    
    output_colors = colors[mask_map]
    pcd.colors = open3d.utility.Vector3dVector(output_colors[::skip] / 256.0)
    
    #pcd.paint_uniform_color(np.array([0,0,1], dtype='float64'))

    #pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud
    pcd.transform(vis_transform)

    # add pcd to visualizer
    vis.add_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    # Close window using esc key
    key = cv2.waitKey(30) & 0xFF
    
    if (key == ord('q')):
        break

vis.destroy_window()
CamL.release()
CamR.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


-1

In [5]:
pip install open3d==0.15.1  # например, для версии 0.15.1

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [6]:
pip install --upgrade pip setuptools wheel

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 262.6 kB/s eta 0:00:07
    --------------------------------------- 0.0/1.8 MB 262.6 kB/s eta 0:00:07
    --------------------------------------- 0.0/1.8 MB 262.6 kB/s eta 0:00:07
   - -------------------------------------- 0.1/1.8 MB 328.6 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.8 MB 350.1 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.8 MB 350.1 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.8 MB 328.4 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.8 MB 328.4 kB/s eta 0:00:06
   --- ------------------------------------ 0.1/1.8 MB 315.4 kB/s eta 0:00:06
   --- ------------------------------------ 0.2/1.8 MB 296.2 kB/s eta 0:00:06
   --- ------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.


In [7]:
pip install open3d

   ---------------------------------------- 0.0/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/69.2 MB ? eta -:--:--
   -----------------